# Testing File
---
_This notebook is for testing new functionality before introducing it into the program._

### Protocol
---
_Pokémon Showdown is accessible via websockets, so I can interact with the website from this file (sending messages, logging in, and so on)._
_Found here: https://github.com/smogon/pokemon-showdown/blob/master/PROTOCOL.md_

_This has been done before in python here: https://github.com/pmariglia/showdown/blob/master/showdown/websocket_client.py_

**Variables I'll need**:
* Websocket UI
* Username and Password
* Format
* Team to use (if applicable)

In [ ]:
import asyncio
import websockets
import requests
import json
import random

# Global used for sending and receiving messages.
socket = await websockets.connect("wss://sim3.psim.us/showdown/websocket")

# Connect to the Showdown server and log in.
async def connectToShowdown(loginUri, testUsername, testPassword):
    # challstr and chall id represent the current user token.
    # we need to retrieve it from the showdown server. When not logged in,
    # the server will send the user their challstr and challid whcih we receive below.
    challid = 0
    challstr = ""

    while(challstr == ""):
        received = await socket.recv()
        msgs = received.split("|")
        if(msgs[1] == "challstr"):
            challid = msgs[2]
            challstr = msgs[3]

    challstr = "|".join([challid, challstr]) # Join these together to form the token to be sent to the server

    # HTTP Request 
    loggedIn = requests.post(
        loginUri,
        data={
            "act": "login",
            "name": testUsername,
            "pass": testPassword,
            "challstr": challstr,
        }
    )

    # Finish logging in by grabbing the "assertion" given to us by showdown in JSON.
    jsonOut = json.loads(loggedIn.text[1:])
    await sendMessage("|/trn "+testUsername+",0,"+jsonOut["assertion"]+"|")

async def sendMessage(message):
    await socket.send([message])
    return await socket.recv()


# Join the queue by sending a search message in a specific format.
# Returns a battle tag, used by other methods to specify what battle room to interact with.
async def joinQueue(format):
    received = await sendMessage("|/search "+format+"|")

    while True:
        received = received.split("|")
        if 'battle' in received[0]:
            return received[0][1:].strip()
        received = await socket.recv()

# Runs the battle logic. When the Model is implemented, will instead make calls to the model for what to do.
# Should also send it the necessary battle info.
async def manageBattle(battle_tag):
    while True:
        received = await socket.recv()
        print(received)
        msgs = received.split("|")

        # Check for a battle
        if 'request' in msgs[1] and len(msgs[2]) > 9:
            requestOutput = json.loads(msgs[2])

            # Make a move
            if 'active' in requestOutput:
                invalidMove = True

                # Pokémon can be locked into moves, so we need to check that the chosen move hasn't been disabled.
                while invalidMove:
                    randomMove = random.choice(requestOutput["active"][0]["moves"])
                    if not randomMove["disabled"]:
                        invalidMove = False
                        await socket.send(battle_tag+'|/choose move '+randomMove["id"]+"|"+str(requestOutput["rqid"]))

            # Mon has fainted - make a switch.
            if 'forceSwitch' in requestOutput:
                invalidMon = True
            
                while invalidMon:
                    randomMon = random.randint(0,5)
                    print(requestOutput["side"]["pokemon"][randomMon])
                    print(requestOutput["side"]["pokemon"][randomMon]["condition"])
                    # Don't send in a fainted mon or try to send in the current one.
                    if(requestOutput["side"]["pokemon"][randomMon]["condition"] != "0 fnt") and (requestOutput["side"]["pokemon"][randomMon]["active"] == False):
                        invalidMon = False
                        await socket.send(battle_tag+'|/choose switch '+str(randomMon+1)+"|"+str(requestOutput["rqid"]))
        
        # Exit once a battle's over.
        if 'win' in msgs[1]:
            return

        

In [ ]:
await connectToShowdown("https://play.pokemonshowdown.com/action.php", "thisisatest12345", "password")
tag = await joinQueue("gen9randombattle")
await manageBattle(tag)